# Adelic Geometric Pairing $Q(g)$ — Verification Notebook

This notebook evaluates
$$Q(g)=\sum_v \langle D_v, W\rangle=\sum_p \langle D_p,W\rangle+\langle D_\infty,W\rangle,$$
for the **band‑limited** test class with
$$W(u)=\frac{1}{\pi}(1-e^{-u})\,g_b(2u),\qquad g(\tau)=\frac{1}{\pi}\int_0^\infty g_b(x)\cos(x\tau)\,dx,$$
and checks the identity
$$Q(g)=P(g)-A(g).$$

**Finite places.** With $D_p(u)=\sum_{k\ge1}(\log p)\,\delta(u-k\log p)$,
\n$\langle D_p,W\rangle=\sum_{k\ge1}(\log p)\,W(k\log p)=\tfrac{1}{\pi}\sum_{k\ge1}(\log p)(1-p^{-k})\,g_b(2k\log p)$.

**Archimedean place.** Using the Maaß–Selberg normalization, the modulus‑line kernel is
$$K_\infty(u)=-\frac{e^{-u}}{1-e^{-2u}}=-\frac{1}{2\sinh u},\qquad \langle D_\infty,W\rangle=\int_0^{\infty}K_\infty(u)W(u)\,du,$$
and this pairing equals $-A(g)$ where
$$A(g)=\frac{1}{2\pi}\int_{\mathbb R} g(\tau)\,\Re\{\psi(i\tau)-\psi(\tfrac12+i\tau)\}\,d\tau.$$

**Provenance / normalizations.** These conventions match the Eisenstein‑phase derivation of the asymmetric explicit formula and its geometric blocks, and the adelic distributional framework for the symmetric side. See Paper 01 for the Eisenstein‑phase route and Maaß–Selberg normalization, Paper 03B for the modulus‑line distributional setup and the dictionary $W(u)$, and Paper 11 for the Mellin–torsion identity and $u$‑side expressions for $A(g)$ used as numerical cross‑checks. :contentReference[oaicite:0]{index=0} :contentReference[oaicite:1]{index=1} :contentReference[oaicite:2]{index=2}

**Tests included.**
1. *Silent primes:* support radius $X<2\log 2$ so $P(g)=0$; expect $Q(g)=-A(g)$.
2. *Single spike:* a narrow window around $2\log 2$; discrete block should match its closed form to machine precision.
3. *Multi‑spike:* a wider radius bump ($X=4$) catching primes $2,3,5,7$ and small powers.

Acceptance thresholds used below:
- $|P_{\text{closed}}-P_{\text{sample}}|<10^{-12}$; 
- $|\langle D_\infty,W\rangle + A_u|<10^{-12}$ and $|\langle D_\infty,W\rangle + A_\tau|<10^{-6}$; 
- $|Q-(P-A_u)|<10^{-12}$; 
- Silent window: $|P|<10^{-12}$ and $|Q+A_u|<10^{-10}$.


In [ ]:
# Core machinery: band-limited tests, local blocks, and verifications

import math
from math import log, exp, pi
import numpy as np
import mpmath as mp

mp.mp.dps = 80  # high precision for stable u-side quadrature
mp.mp.pretty = True

def sieve_primes_upto(n: int):
    if n < 2:
        return []
    sieve = bytearray(b"\x01") * (n + 1)
    sieve[0:2] = b"\x00\x00"
    for p in range(2, int(n**0.5) + 1):
        if sieve[p]:
            step = p
            start = p*p
            sieve[start:n+1:step] = b"\x00" * ((n - start)//step + 1)
    return [i for i in range(2, n + 1) if sieve[i]]

def simpson_weights(n: int, dx: float):
    assert n % 2 == 1, "Simpson requires odd number of nodes"
    w = np.ones(n)
    w[1:-1:2] = 4
    w[2:-1:2] = 2
    w *= dx/3.0
    return w

# C^\infty compactly-supported bumps for gb
def smooth_bump_radius(X: float, amp: float = 1.0):
    X = float(X)
    invX2 = 1.0/(X*X)
    def gb(x: float):
        ax = abs(x)
        if ax >= X:
            return mp.mpf('0')
        t = 1 - (ax*ax)*invX2
        return mp.mpf(amp) * mp.e**(-1.0/t)
    return gb

def smooth_bump_centered(c: float, w: float, amp: float = 1.0):
    c = float(c); w = float(w)
    invw2 = 1.0/(w*w)
    def gb(x: float):
        y = abs(x)
        d = abs(y - c)
        if d >= w:
            return mp.mpf('0')
        t = 1 - (d*d)*invw2
        return mp.mpf(amp) * mp.e**(-1.0/t)
    return gb

# Dictionary W(u) and archimedean kernel K_\infty(u)
def W_of_u(u, gb):
    if u < 0:
        return mp.mpf('0')
    return (mp.mpf('1')/mp.pi) * (1 - mp.e**(-u)) * gb(2*u)

def K_inf(u):
    """K_\infty(u) = -1/(2*sinh u) with small-u series for stability."""
    u = mp.mpf(u)
    au = abs(u)
    if au < mp.mpf('1e-12'):
        if u == 0:
            return mp.ninf
        # 1/(2*sinh u) ~ 1/(2u) - u/12 + 7 u^3/720
        return -(1/(2*u) - u/12 + 7*(u**3)/720)
    return - 1 / (2 * mp.sinh(u))

def Dinf_pairing(gb, X: float):
    """⟨D_\infty, W⟩ = ∫_0^{X/2} K_\infty(u) W(u) du; near 0 use u=t^2."""
    Umax = X/2.0
    def f(u):
        return K_inf(u) * W_of_u(u, gb)
    eps = mp.mpf('1e-10')
    def near0(t):
        if t == 0:
            return mp.mpf('0')
        u = t*t
        return f(u) * 2*t
    val = mp.mpf('0')
    if Umax > eps:
        val += mp.quad(near0, [0, mp.sqrt(eps)])
        val += mp.quad(f, [eps, Umax])
    else:
        val += mp.quad(near0, [0, mp.sqrt(Umax)])
    return val

# Finite-prime block two ways (for equality test)
def P_closed(gb, X: float):
    Umax = X/2.0
    pmax = int(math.floor(math.exp(Umax)) + 2)
    primes = sieve_primes_upto(pmax)
    total = mp.mpf('0')
    for p in primes:
        lp = math.log(p)
        kmax = int(math.floor(Umax / lp))
        for k in range(1, kmax+1):
            total += (mp.mpf('1')/mp.pi) * lp * (1 - p**(-k)) * gb(2*k*lp)
    return total

def P_sample(gb, X: float):
    Umax = X/2.0
    pmax = int(math.floor(math.exp(Umax)) + 2)
    primes = sieve_primes_upto(pmax)
    total = mp.mpf('0')
    for p in primes:
        lp = math.log(p)
        kmax = int(math.floor(Umax / lp))
        for k in range(1, kmax+1):
            total += lp * W_of_u(k*lp, gb)
    return total

# A(g): u-side closed form and τ-side digamma integral
def A_u_side(gb, X: float):
    return (mp.mpf('1')/(2*mp.pi)) * mp.quad(lambda t: gb(t)/(mp.e**(t/2.0) + 1.0), [0, X])

def g_from_gb_on_grid(taus, x_nodes, gb_vals, w_x):
    A = (w_x * gb_vals)/math.pi
    return A @ np.cos(np.outer(x_nodes, taus))

def KA_nozero(taus):
    vals = []
    for t in taus:
        psi1 = mp.digamma(mp.mpc(0, t))
        psi2 = mp.digamma(mp.mpc(0.5, t))
        vals.append(float(mp.re(psi1 - psi2)))
    return np.array(vals, dtype=np.float64)

def A_tau_side(gb, X: float, T_tau: float = 40.0, n_tau: int = 4001, n_x: int = 2001, tau0: float = 1e-6):
    nx = int(n_x)
    if nx % 2 == 0: nx += 1
    x_nodes = np.linspace(0.0, float(X), nx)
    dx = float(X)/(nx-1)
    w_x = simpson_weights(nx, dx)
    gb_vals = np.array([float(gb(xx)) for xx in x_nodes], dtype=np.float64)

    ntau = int(n_tau)
    if ntau % 2 == 0: ntau += 1
    taus = np.linspace(float(tau0), float(T_tau), ntau)
    dτ = (float(T_tau) - float(tau0))/(ntau-1)
    w_tau = simpson_weights(ntau, dτ)

    g_vals  = g_from_gb_on_grid(taus, x_nodes, gb_vals, w_x)
    KA_vals = KA_nozero(taus)
    return (1.0/math.pi) * float(np.dot(w_tau, g_vals*KA_vals))


In [ ]:
# Define tests, run them, show a summary table, and assert acceptance tests
import pandas as pd

# 1) Silent primes: X < 2 log 2
X_silent = 1.30  # 2*log 2 ≈ 1.38629
gb_silent = smooth_bump_radius(X_silent, amp=1.0)

# 2) Single spike at 2 log 2
x0 = 2.0*math.log(2.0); w = 0.03
gb_single = smooth_bump_centered(x0, w, amp=1.0)
X_single = x0 + 1.1*w

# 3) Multi-spike up to ~e^2 (primes 2,3,5,7 and some powers)
X_multi = 4.0
gb_multi = smooth_bump_radius(X_multi, amp=1.0)

tests = [
    ("Silent primes (X < 2 log 2)", gb_silent, X_silent),
    ("Single spike at 2 log 2", gb_single, X_single),
    ("Multi-spike (up to ~e^2)", gb_multi, X_multi),
]

rows = []
for name, gb, X in tests:
    Pcl = P_closed(gb, X)
    Psm = P_sample(gb, X)
    Dinf = Dinf_pairing(gb, X)
    Au   = A_u_side(gb, X)
    Atau = mp.mpf(A_tau_side(gb, X, T_tau=40.0, n_tau=4001, n_x=2001, tau0=1e-6))
    Q    = Psm + Dinf
    rows.append({
        "test": name,
        "X_support": X,
        "P_closed": float(Pcl),
        "P_sample": float(Psm),
        "|ΔP|": float(abs(Pcl-Psm)),
        "⟨D∞,W⟩": float(Dinf),
        "A_u": float(Au),
        "A_tau": float(Atau),
        "|⟨D∞,W⟩+A_u|": float(abs(Dinf+Au)),
        "|⟨D∞,W⟩+A_tau|": float(abs(Dinf+Atau)),
        "Q": float(Q),
        "P-A_u": float(Pcl - float(Au)),
        "|Q-(P-A_u)|": float(abs(Q-(Pcl - float(Au))))
    })

df = pd.DataFrame(rows, columns=[
    "test","X_support","P_closed","P_sample","|ΔP|","⟨D∞,W⟩","A_u","A_tau",
    "|⟨D∞,W⟩+A_u|","|⟨D∞,W⟩+A_tau|","Q","P-A_u","|Q-(P-A_u)|"
])
display(df)

# Acceptance tests
tol_P       = 1e-12
tol_Au      = 1e-12
tol_Atau    = 1e-6
tol_identity= 1e-12
tol_silentQ = 1e-10

# 1) Discrete block equality
assert (df["|ΔP|"] < tol_P).all(), "Discrete block mismatch: P_closed vs P_sample exceeds tolerance"

# 2) Archimedean pairing vs A(g)
assert (df["|⟨D∞,W⟩+A_u|"] < tol_Au).all(), "u-side A(g) mismatch with ⟨D∞,W⟩"
assert (df["|⟨D∞,W⟩+A_tau|"] < tol_Atau).all(), "τ-side A(g) mismatch with ⟨D∞,W⟩ (looser tol)"

# 3) Main identity Q = P - A
assert (df["|Q-(P-A_u)|"] < tol_identity).all(), "Main identity Q = P - A(u-side) failed"

# 4) Silent window: P≈0 and Q≈-A
row_silent = df[df["test"].str.contains("Silent")].iloc[0]
assert abs(row_silent["P_closed"]) < tol_P, "Silent window: P(g) not ~ 0"
assert abs(row_silent["Q"] + row_silent["A_u"]) < tol_silentQ, "Silent window: Q(g) ≠ -A(g)"

print("All acceptance tests PASSED.")
print("Typical tolerances:")
print(f"  |ΔP| ≤ {tol_P},  |⟨D∞,W⟩+A_u| ≤ {tol_Au},  |⟨D∞,W⟩+A_tau| ≤ {tol_Atau},  |Q-(P-A_u)| ≤ {tol_identity}")

print("\nNormalization crib (Maaß–Selberg 1/(2π) on spectral side):")
print("  W(u)=(1/π)(1-e^{-u}) g_b(2u);  D_p(u)=∑_{k≥1}(log p) δ(u-k log p);")
print("  K_∞(u)=-e^{-u}/(1-e^{-2u});  A(g)=(1/(2π))∫ g(τ) Re{ψ(iτ)-ψ(1/2+iτ)} dτ.")
print("  These match the Eisenstein‑phase normalization and the adelic dictionary.")
print("  References inside this notebook header.  (Papers 01, 03B, 11)")
